In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd '/content/drive/My Drive/ProtonX/Kaggle 03'

/content/drive/My Drive/ProtonX/Kaggle 03


In [3]:
#ls

# Data

In [4]:
dataset_folder = './Datasets'

In [5]:
#!unzip -q Archive.zip -d $dataset_folder

In [6]:
train_folder = './Datasets/train'
test_folder = './Datasets/Test/'

In [7]:
img_height = 224
img_width = 224
batch_size = 32

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
#Add out data-augmentation parameters to ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1./127.5, 
                                     validation_split=0.2,
                                     rotation_range=40, 
                                     width_shift_range=0.2, 
                                     height_shift_range=0.2, 
                                     shear_range=0.2, 
                                     zoom_range=0.3, 
                                     horizontal_flip=True, 
                                     fill_mode='nearest')

validation_generator = ImageDataGenerator(rescale=1./127.5, validation_split=.2)
#Note that the validation data should not be augmented
test_generator = ImageDataGenerator(rescale=1./127.5)


In [10]:
train_ds = train_generator.flow_from_directory(
  train_folder,
  target_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=True,
  interpolation="bilinear",
  subset='training',
  class_mode='sparse')

valid_ds = validation_generator.flow_from_directory(
  train_folder,
  target_size=(img_height, img_width),
  batch_size=batch_size,
  shuffle=False,
  subset='validation',
  interpolation="bilinear",
  class_mode='sparse')

Found 4828 images belonging to 5 classes.
Found 1206 images belonging to 5 classes.


In [11]:
test_ds = test_generator.flow_from_directory(
    directory=r"./Datasets/Test/",
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42)

Found 1079 images belonging to 1 classes.



# Build Model


In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.applications import VGG16
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, Resizing
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import tensorflow_hub as hub
from keras.layers.advanced_activations import LeakyReLU

from keras.applications import InceptionV3, ResNet50, VGG16
#from tensorflow.keras.applications.

In [13]:
DESIRED_ACCURACY = 0.985

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}) :
        if(logs.get('acc') is not None and logs.get('acc') >= DESIRED_ACCURACY) :
            print('\nReached 99.9% accuracy so cancelling training!')
            self.model.stop_training = True

callbacks = myCallback()

## Resnet

In [ ]:
# resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4" 

# model_resnet = Sequential([
#                            Resizing(224, 224, input_shape=(224, 224, 3)),
#                            hub.KerasLayer(resnet_url, input_shape=(224, 224, 3), trainable=False),
#                            #GlobalAveragePooling2D(),
#                            #BatchNormalization(),
#                            #Flatten(),
#                            #Dense(2048, activation='relu'),
#                            #Dropout(0.5),
#                            #Dense(1024, activation='relu'),
#                            #Dropout(0.3),
#                            Dense(5, activation="softmax")

# ])

# model_resnet.summary()

In [ ]:
# model_resnet.compile(loss='sparse_categorical_crossentropy',
#               optimizer=Adam(0.0001),
#               metrics=['accuracy'])

In [ ]:
epochs=50

In [ ]:
# model_resnet.fit_generator(
#     generator=train_ds, 
#     steps_per_epoch=150,   # -> 106 # images 4828 = steps * batch_size = 150 * 32 
#     epochs=epochs, 
#     validation_steps=37, # -> 26 # images 1206 = steps * batch_size = 37 * 32
#     validation_data=valid_ds, 
#     verbose=2
# )

In [ ]:
#resnet.save('resnet.h5')

## MobileNet

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
module_selection = ("mobilenet_v2_100_224", 224) #@param ["(\"mobilenet_v2_100_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)

In [ ]:
#mobilenet_url = " https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4" 
#mobilenet_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4" 
mobilenet = Sequential([
                        tf.keras.layers.InputLayer(input_shape=(224, 224) + (3,)),
                        hub.KerasLayer(MODULE_HANDLE, trainable=False),
                        
                        #GlobalAveragePooling2D(),
                        Dropout(0.5),
                        #BatchNormalization(),
                        Flatten(),
                        Dense(512, activation='relu'),
                        Dropout(0.5),
                        Dense(100, activation='relu'),
                        Dropout(0.3),
                        Dense(5, activation="softmax")

])
mobilenet.build((None,)+(224, 224)+(3,))
mobilenet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               128100    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 505       
Total params: 2,386,589
Trainable params: 128,605
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
mobilenet.compile(loss='sparse_categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

In [ ]:
steps_per_epoch = train_ds.samples // train_ds.batch_size
validation_steps = valid_ds.samples // valid_ds.batch_size
hist = mobilenet.fit(
    train_ds,
    epochs=30, steps_per_epoch=steps_per_epoch,
    validation_data=valid_ds,
    validation_steps=validation_steps,
    callbacks=callbacks).history

Epoch 1/30
75/75 [==============================] - 71s 951ms/step - loss: 0.6623 - accuracy: 0.7538 - val_loss: 0.3631 - val_accuracy: 0.8733
Epoch 2/30
75/75 [==============================] - 71s 952ms/step - loss: 0.3447 - accuracy: 0.8810 - val_loss: 0.3449 - val_accuracy: 0.8707
Epoch 3/30
75/75 [==============================] - 71s 951ms/step - loss: 0.2589 - accuracy: 0.9099 - val_loss: 0.3452 - val_accuracy: 0.8715
Epoch 4/30
75/75 [==============================] - 70s 937ms/step - loss: 0.2132 - accuracy: 0.9326 - val_loss: 0.3541 - val_accuracy: 0.8698
Epoch 5/30
75/75 [==============================] - 70s 938ms/step - loss: 0.1833 - accuracy: 0.9391 - val_loss: 0.3970 - val_accuracy: 0.8507
Epoch 6/30
75/75 [==============================] - 71s 948ms/step - loss: 0.1532 - accuracy: 0.9536 - val_loss: 0.3484 - val_accuracy: 0.8828
Epoch 7/30
75/75 [==============================] - 70s 931ms/step - loss: 0.1433 - accuracy: 0.9524 - val_loss: 0.3784 - val_accuracy: 0.8707

In [ ]:
# mobilenet.fit_generator(
#     generator=train_ds, 
#     steps_per_epoch=150,   # -> 106 # images 4828 = steps * batch_size = 150 * 32 
#     epochs=epochs, 
#     validation_steps=37, # -> 26 # images 1206 = steps * batch_size = 37 * 32
#     validation_data=valid_ds, 
#     verbose=2
# )

In [ ]:
mobilenet.save('MobileNetSGD.h5')



In [ ]:
IMG_SHAPE = (224, 224, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
                            base_model,
                            GlobalAveragePooling2D(),
                            #Dense(512, activation='relu'),
                            #Dropout(0.5),
                            Dense(100, activation='relu'),
                            Dropout(0.3),
                            Dense(5, activation="softmax")
  
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
              metrics=['accuracy'])
steps_per_epoch = train_ds.samples // train_ds.batch_size
validation_steps = valid_ds.samples // valid_ds.batch_size
hist = model.fit(
    train_ds,
    epochs=30, steps_per_epoch=steps_per_epoch,
    validation_data=valid_ds,
    validation_steps=validation_steps).history

In [ ]:
model.save('Global.h5')

# InceptionNet

In [15]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [16]:
local_weight_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [17]:
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False)

base_model.load_weights(local_weight_file)
# last_layer = base_model.get_layer('mixed7')
# last_output = last_layer.output

# for layer in base_model.layers:
#   layer.trainable=False

for layer in base_model.layers[:249]:
      layer.trainable = False
    
    # Setting last layers as trainable | Unfreezing later layers
for layer in base_model.layers[249:]:
      layer.trainable = True

#for layer in baseModel.layers[:-4]:
 # layer.trainable = False
model1 = Sequential([
                     #tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1),
                     base_model,
                     Dropout(0.4),
                     #
                     GlobalAveragePooling2D(),
                     #Flatten(),
                    #  Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)),
                    #  LeakyReLU(0.02),
                     #Dropout(0.4),
                     Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)),
                     #LeakyReLU(0.02),
                     #Dropout(0.3),
                     Dense(5, activation='softmax', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01))
])
model1.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 2048)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 22,328,613
Trainable params: 11,640,709
Non-trainable params: 10,687,904
_________________________________________________________________


In [ ]:
model1.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.0001),
               metrics=['accuracy'])

steps_per_epoch = train_ds.samples // train_ds.batch_size
validation_steps = valid_ds.samples // valid_ds.batch_size
hist = model1.fit(
    train_ds,
    epochs=20, steps_per_epoch=steps_per_epoch,
    validation_data=valid_ds,
    validation_steps=validation_steps,
    callbacks=callbacks
    ).history

Epoch 1/20
150/150 [==============================] - 1622s 11s/step - loss: 5.0955 - accuracy: 0.7467 - val_loss: 4.2803 - val_accuracy: 0.8387
Epoch 2/20
150/150 [==============================] - 78s 517ms/step - loss: 3.7125 - accuracy: 0.8609 - val_loss: 3.2154 - val_accuracy: 0.8801
Epoch 3/20
150/150 [==============================] - 75s 499ms/step - loss: 2.7549 - accuracy: 0.9022 - val_loss: 2.4676 - val_accuracy: 0.8708
Epoch 4/20
150/150 [==============================] - 74s 495ms/step - loss: 2.0614 - accuracy: 0.9291 - val_loss: 1.9472 - val_accuracy: 0.8623
Epoch 5/20
150/150 [==============================] - 74s 496ms/step - loss: 1.5719 - accuracy: 0.9441 - val_loss: 1.5427 - val_accuracy: 0.8750
Epoch 6/20
150/150 [==============================] - 74s 494ms/step - loss: 1.2253 - accuracy: 0.9502 - val_loss: 1.3201 - val_accuracy: 0.8606
Epoch 7/20
150/150 [==============================] - 75s 498ms/step - loss: 0.9857 - accuracy: 0.9527 - val_loss: 1.0858 - val_ac

In [ ]:
model1.save('2.h5')

# VGG16

In [14]:
from tensorflow.keras import regularizers

In [ ]:
baseModel = VGG16(weights='imagenet', include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

# ADD some layers
# Lấy output của ConvNet trong VGG16
# freez VGG model
for layer in baseModel.layers[:-4]:
  layer.trainable = False
VGG_16 = Sequential([
                     baseModel,
                     Dropout(0.4),
                     #
                     GlobalAveragePooling2D(),
                     #Flatten(),
                    #  Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)),
                    #  LeakyReLU(0.02),
                     #Dropout(0.4),
                     Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)),
                     LeakyReLU(0.02),
                     Dropout(0.3),
                     Dense(5, activation='softmax', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01))
])
VGG_16.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 512)         0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_______________________________________________________

In [ ]:
opt = RMSprop(0.0001)
VGG_16.compile(opt, 'sparse_categorical_crossentropy', ['accuracy'])
numOfEpoch = 25
steps_per_epoch = train_ds.samples // train_ds.batch_size
validation_steps = valid_ds.samples // valid_ds.batch_size
H = VGG_16.fit(
    train_ds,
    epochs=20, steps_per_epoch=steps_per_epoch,
    validation_data=valid_ds,
    validation_steps=validation_steps,
    callbacks=callbacks).history

Epoch 1/20
150/150 [==============================] - 84s 562ms/step - loss: 3.8848 - accuracy: 0.6791 - val_loss: 2.9643 - val_accuracy: 0.8488
Epoch 2/20
150/150 [==============================] - 84s 560ms/step - loss: 2.5608 - accuracy: 0.8163 - val_loss: 2.0541 - val_accuracy: 0.8767
Epoch 3/20
150/150 [==============================] - 84s 557ms/step - loss: 1.7966 - accuracy: 0.8561 - val_loss: 1.5020 - val_accuracy: 0.8944
Epoch 4/20
150/150 [==============================] - 83s 552ms/step - loss: 1.3251 - accuracy: 0.8822 - val_loss: 1.2349 - val_accuracy: 0.8733
Epoch 5/20
150/150 [==============================] - 83s 551ms/step - loss: 1.0638 - accuracy: 0.8974 - val_loss: 1.0161 - val_accuracy: 0.8792
Epoch 6/20
150/150 [==============================] - 83s 553ms/step - loss: 0.8894 - accuracy: 0.9191 - val_loss: 0.9463 - val_accuracy: 0.8826
Epoch 7/20
150/150 [==============================] - 85s 568ms/step - loss: 0.7962 - accuracy: 0.9208 - val_loss: 0.8546 - val_ac

KeyboardInterrupt: ignored

In [ ]:
VGG_16.save("VGG_16_0.41.h5")

In [ ]:
# unfreeze some last CNN layer:
for layer in baseModel.layers[15:]:
    layer.trainable = True

numOfEpoch = 35
opt = Adam(0.0001)
VGG_16.compile(opt, 'sparse_categorical_crossentropy', ['accuracy'])
H = VGG_16.fit(
    train_ds,
    epochs=10, steps_per_epoch=steps_per_epoch,
    validation_data=valid_ds,
    validation_steps=validation_steps)

Epoch 1/10
150/150 [==============================] - 82s 548ms/step - loss: 0.3264 - accuracy: 0.9933 - val_loss: 0.5656 - val_accuracy: 0.9003
Epoch 2/10
150/150 [==============================] - 83s 551ms/step - loss: 0.3215 - accuracy: 0.9912 - val_loss: 0.5377 - val_accuracy: 0.9062
Epoch 3/10
150/150 [==============================] - 81s 543ms/step - loss: 0.3143 - accuracy: 0.9921 - val_loss: 0.5442 - val_accuracy: 0.9029
Epoch 4/10
150/150 [==============================] - 81s 543ms/step - loss: 0.3009 - accuracy: 0.9944 - val_loss: 0.5719 - val_accuracy: 0.8936
Epoch 5/10
150/150 [==============================] - 82s 547ms/step - loss: 0.2979 - accuracy: 0.9946 - val_loss: 0.5498 - val_accuracy: 0.8936
Epoch 6/10
150/150 [==============================] - 82s 549ms/step - loss: 0.3035 - accuracy: 0.9917 - val_loss: 0.5104 - val_accuracy: 0.9223
Epoch 7/10
150/150 [==============================] - 82s 549ms/step - loss: 0.2869 - accuracy: 0.9958 - val_loss: 0.5025 - val_ac

# Predict


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
save_model = tf.keras.models.load_model('2.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
test_df = pd.read_csv('./sample_submission.csv')

In [ ]:
STEP_SIZE_TEST=test_ds.n//test_ds.batch_size
test_ds.reset()
pred=save_model.predict_generator(test_ds,
steps=STEP_SIZE_TEST,
verbose=1)

1079/1079 [==============================] - 12s 11ms/step


In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_ds.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [k for k in predicted_class_indices]

# Submit


In [ ]:
filenames=test_ds.filenames
results=pd.DataFrame({"File Name":filenames,
                      "Label":predictions})
results.head()

,File Name,Label
0,test/10043234166_e6dd915111_n.jpg,1
1,test/10128546863_8de70c610d.jpg,4
2,test/10172636503_21bededa75_n.jpg,0
3,test/10200780773_c6051a7d71_n.jpg,0
4,test/1022552036_67d33d5bd8_n.jpg,3


In [ ]:
results['File Name'] = results['File Name'].apply(lambda x: x[5:])
results.head()

,File Name,Label
0,10043234166_e6dd915111_n.jpg,1
1,10128546863_8de70c610d.jpg,4
2,10172636503_21bededa75_n.jpg,0
3,10200780773_c6051a7d71_n.jpg,0
4,1022552036_67d33d5bd8_n.jpg,3


In [ ]:
results.to_csv('2.csv',index=False)